In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [2]:
import sklearn
sklearn.__version__

'1.5.0'

In [3]:
!python -V

Python 3.12.2


In [4]:
import pickle
import pandas as pd

In [5]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [6]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [7]:
year = 2023
month = 3

In [8]:
df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:02d}-{month:02d}.parquet')

In [9]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [10]:
y_pred.std()

6.247488852238703

In [11]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [12]:
output_file = 'output/result.parquet'

In [13]:
df_result = pd.DataFrame()

In [14]:
df_result['ride_id'] = df['ride_id']
df_result['pred'] = y_pred

In [15]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [16]:
!ls -l output

total 67036
-rw-rw-rw- 1 codespace codespace 68641674 Jun 16 11:38 result.parquet


In [17]:
!jupyter nbconvert --to script starter.ipynb

[NbConvertApp] Converting notebook starter.ipynb to script
[NbConvertApp] Writing 1570 bytes to starter.py
